In [3]:
# standard python utilities
import os
import sys
from os.path import basename, dirname, join, exists
import glob
import pandas as pd
import numpy as np
import time
from scipy.stats import gmean


# standard geospatial python utilities
# import pyproj # for converting proj4string
import shapely
import shapefile
import geopandas as gpd
from osgeo import gdal
import rasterio

import flopy

In [4]:
ext_dir = 'F:/WRDAPP'
c_dir = 'C:/WRDAPP'
if os.path.exists(ext_dir):
    loadpth = ext_dir 
elif os.path.exists(c_dir):
    loadpth = c_dir 
loadpth +=  '/GWFlowModel/Cosumnes/Stream_seepage'
model_nam = 'inset_oneto_denier'

model_ws = join(loadpth,model_nam)

m = flopy.modflow.Modflow.load('MF.nam', model_ws= model_ws, 
                                exe_name='mf-owhm.exe', version='mfnwt')



In [5]:
all_model_ws = join(loadpth, 'parallel_oneto_denier')

# Copy files independent of geology

In [ ]:
# directly copy files not impacted by changing geology
pks = ['nam','dis','nwt','bas','oc','evt', 'gage', 'hob', 'tab','wel','bath']
files = [glob.glob(model_ws+'/*'+p)[0] for p in pks]


In [ ]:
import shutil, os

for n in np.arange(0,100).astype(str):
    for f in files:
        folder = '/realization'+ n.zfill(3)+'/'
        os.makedirs(all_model_ws+folder,exist_ok=True)
        shutil.copy(f, model_ws+folder)

# Create files dependent on geology

In [ ]:
model_ws

mf_tprogs_dir = gwfm_dir+'/UPW_data/tprogs_final'+tprogs_id+'/'
tprogs_files = glob.glob(mf_tprogs_dir+'*')


In [ ]:

gwfm_dir = 'C:/Users/ajcalder/Box/research_cosumnes/GWFlowModel/'
proj_dir = gwfm_dir+'Levee_setback/'

# save modflow workspace file to WRDAPP sub folder to improve runtime calculations
loadpth = 'F:/WRDAPP/GWFlowModel/Cosumnes/levee_setback/streamflow/'
# load model with only DIS to reduce load time
# the model will run off of the .nam file connection so flopy doesn't need them
all_model_ws = loadpth + '/setback_streamflow'

tprogs_fxn_dir = 'C:/Users/'+getuser()+'/Documents/GitHub/CosumnesRiverRecharge/tprogs_utilities'
if tprogs_fxn_dir not in sys.path:
    sys.path.append(tprogs_fxn_dir)
        
for model_nam in basename(all_model_ws)+pd.Series([0,33,89]).astype(str).str.zfill(3).values:
    # in the case this runs in the basefolder then the current working directory
    # is the same as the all working directory - useful now with weird naming between realizations
    # if basename(model_ws).__contains__('historical'):
    #     all_model_ws = model_ws
    model_ws = loadpth +'/'+model_nam
    m = flopy.modflow.Modflow.load('MF.nam', model_ws=model_ws, 
                                    exe_name='mf-owhm.exe', version='mfnwt')

    nrow = m.dis.nrow
    ncol = m.dis.ncol
    nlay = m.dis.nlay

    botm = m.dis.botm.array
    # num_tprogs = 120 (max available below levelling), upscaling
    max_num_layers =148 # based on thickness from -6m (1 m below DEM min) to -80m
    upscale = 8
    num_tprogs = int(max_num_layers/upscale)

    ###############################################################################
    ## LPF Package ##
    deep_geology = np.loadtxt(model_ws+'/input_data/deep_geology.tsv', delimiter ='\t')
    deep_geology = np.reshape(deep_geology, (m.dis.nlay,m.dis.nrow,m.dis.ncol))
    # initial guess for hydraulic parameters
    params = pd.read_csv(model_ws+'/ZonePropertiesInitial.csv')
    params = params.set_index('Zone')
    # convert from m/s to m/d
    params['K_m_d'] = params.K_m_s * 86400    

    import tprogs_cleaning as tc

    # load tprogs facies array and convert to values based on params
    masked_tprogs = np.reshape(np.loadtxt(model_ws+'/input_data/tprogs_facies_array.tsv', delimiter='\t'), (320,100,230))
    K, Sy, Ss= tc.int_to_param(masked_tprogs, params)

    Kx_upscaled = np.zeros((num_tprogs,nrow,ncol))
    Kz_upscaled = np.zeros((num_tprogs,nrow,ncol))
    Sy_upscaled = np.zeros((num_tprogs,nrow,ncol))
    Ss_upscaled = np.zeros((num_tprogs,nrow,ncol))

    for k in np.arange(1,num_tprogs+1):
        # calculate upscale from bottom up
        Kx_upscaled[-k,:,:] = np.nanmean(K[(-k*upscale):(-k*upscale-upscale):-1,:,:],axis=0)
        Kz_upscaled[-k,:,:] = hmean(K[(-k*upscale):(-k*upscale-upscale):-1,:,:],axis=0)
        Sy_upscaled[-k,:,:] = np.nanmean(Sy[(-k*upscale):(-k*upscale-upscale):-1,:,:],axis=0)
        Ss_upscaled[-k,:,:] = np.nanmean(Ss[(-k*upscale):(-k*upscale-upscale):-1,:,:],axis=0)

    # allocate arrays for Kx, Ky
    hk = np.zeros(botm.shape)
    vka = np.zeros(botm.shape)
    sy = np.zeros(botm.shape)
    ss = np.zeros(botm.shape)
    # # take of 2 for the bottom layers and 1 for the unsat zone layer up top
    hk[1:-2,:,:] = Kx_upscaled
    vka[1:-2,:,:] = Kz_upscaled
    sy[1:-2,:,:] = Sy_upscaled
    ss[1:-2,:,:] = Ss_upscaled

    tprogs_info = [80, -80, 320]
    top = m.dis.top.array
    bot1 = m.dis.botm.array[0,:,:]
    # set parameters based on upscaled unsaturated zone
    hk[0,:,:] = np.mean(tc.get_tprogs_for_elev(K, top, bot1,tprogs_info),axis=0)
    vka[0,:,:] = hmean(tc.get_tprogs_for_elev(K, top, bot1,tprogs_info),axis=0)
    sy[0,:,:] = np.mean(tc.get_tprogs_for_elev(Sy, top, bot1,tprogs_info),axis=0)
    ss[0,:,:] = np.mean(tc.get_tprogs_for_elev(Ss, top, bot1,tprogs_info),axis=0)


    # set values for second to bottom layer, Laguna formation
    hk[-2,:,:] = params.loc[5,'K_m_d']
    vka[-2,:,:] = params.loc[5,'K_m_d']/100 # assume 1/100 for Kx to Kz
    sy[-2,:,:] = params.loc[5,'Sy']
    ss[-2,:,:] = params.loc[5,'Ss']

    # the deep_geology array shows where the mehrten formation comes out of the surface
    hk[deep_geology[:,:,:].astype(bool)] = params.loc[6,'K_m_d']
    vka[deep_geology[:,:,:].astype(bool)] = params.loc[6,'K_m_d']/100 # assume 1/100 for Kx to Kz
    sy[deep_geology[:,:,:].astype(bool)] = params.loc[6,'Sy']
    ss[deep_geology[:,:,:].astype(bool)] = params.loc[6,'Ss']

    # set values for bottom layer, Mehrten formation
    hk[-1,:,:] = params.loc[6,'K_m_d']
    vka[-1,:,:] = params.loc[6,'K_m_d']/100 # assume 1/100 for Kx to Kz
    sy[-1,:,:] = params.loc[6,'Sy']
    ss[-1,:,:] = params.loc[6,'Ss']

    # 0—indicates VKA is vertical hydraulic conductivity
    layvka = 0
    # no defined anisotropy between kx and ky, could set a value based on stream deposition
    hani = 1 

    # LAYTYP MUST BE GREATER THAN ZERO WHEN IUZFOPT IS 2
    # 0 is confined, >0 convertible, <0 convertible unless the THICKSTRT option is in effect
    # laytyp = [1,1,1,0,0,0,0,0]
    laytyp = np.zeros(m.dis.nlay,dtype=int).tolist()

    laywet=laytyp[:]
    # Laywet must be 0 if laytyp is confined laywet = [1,1,1,1,1]
    # laywet = 1 means layers can be rewetted.
    #ipakcb = 53 means cell-by-cell budget is saved because it is non zero (default is 53)
    # indicates that variable Ss and SS parameters are read as storage coefficient rather than specific storage. (default is False).
    # removed sy while ss is active as storage coefficient rather than specific storage
    # ss= sy for storage coefficient run
    # using storage coefficient drastically increased run time from 10 to 25 minutes, storagecoefficient =True
    lpf = flopy.modflow.ModflowLpf(model = m, hk =hk, chani = 0, hani = hani,
                                   layvka = layvka, vka = vka, ss=ss, sy=sy,
                                   laytyp=laytyp, laywet = laywet, ipakcb=53)
    # overwrite the previous lpf file with updated version
    lpf.write_file()

    print('LPF done')
    #################################################################
    ## SFR K update ##
    sfr = m.sfr

    sfr_rows = sfr.reach_data['i']
    sfr_cols = sfr.reach_data['j']
    strbd_thick = 4
    top = m.dis.top.array
    bot_str_arr = m.dis.top.array- strbd_thick
    # get_tprogs_for_elev(K, m_c.dis.top.array, m_c.dis.top.array- np.linspace(1,4,m_c.dis.ncol), rows = sfr_rows, cols = sfr_cols)
    strbd_tprogs = tc.get_tprogs_for_elev(K, top, bot_str_arr, tprogs_info,rows = sfr_rows, cols = sfr_cols)
    sfr_K = gmean(strbd_tprogs,axis=0)/100 # divide by 100 to ease convergence, but variability is still there
    sfr.reach_data.strhc1 = sfr_K

    sfr.write_file()


    print('SFR done')

    #################################################################
    ## GHB, RCH, WEL scaling factors ##
    scaling_factors = pd.read_csv(model_ws+'/GHB_UZF_WEL_scaling.csv', delimiter = ',')

    ###############################################################################
    ## GHB Package ##
    # join top and botm for easier array referencing for elevations
    top_botm = np.zeros((m.dis.nlay+1,m.dis.nrow,m.dis.ncol))
    top_botm[0,:,:] = m.dis.top.array
    top_botm[1:,:,:] = m.dis.botm.array
    # load ghb dataframes
    ghbse_spd = pd.read_csv(model_ws+'/input_data/ghbse_spd.csv')
    ghbnw_spd = pd.read_csv(model_ws+'/input_data/ghbnw_spd.csv')
    ghbdelta_spd = pd.read_csv(model_ws+'/input_data/ghbdelta_spd.csv')
    # edit GHB conductance by scaling with the new K vs original
    # ghb_hk_nw = scaling_factors.loc[0,'K_nw']*86400
    # ghb_hk_se = scaling_factors.loc[0,'K_se']*86400
    K_delta = scaling_factors.loc[0,'K_delta']*86400

    # only need to recalculate conductance in ucode
    def recalc_cond(i,j,k,hk):
        distance = 5000
        delr = m.dis.delr.array.mean()
        cond = hk*(top_botm[k,i,j]-top_botm[k+1,i,j])*delr/distance
        return(cond)

    ##########################################################
    ## NW, SE GHB ##

    ghb_hk_se = hk[ghbse_spd.k.values, ghbse_spd.i.values,ghbse_spd.j.values]
    ghbse_spd.cond = recalc_cond(ghbse_spd.i.values,ghbse_spd.j.values,ghbse_spd.k.values, ghb_hk_se)
    ghb_hk_nw = hk[ghbnw_spd.k.values, ghbnw_spd.i.values,ghbnw_spd.j.values]
    ghbnw_spd.cond = recalc_cond(ghbnw_spd.i.values,ghbnw_spd.j.values,ghbnw_spd.k.values, ghb_hk_nw)
    ghbdelta_spd.cond = recalc_cond(ghbdelta_spd.i.values,ghbdelta_spd.j.values,ghbdelta_spd.k.values, K_delta)

    # lay, row, col for delta ghb
    zxy = ghbdelta_spd.values[:,:3].astype(int)
    # drop any delta ghb cells where cell bottom is below sea level
    ghbdn_spd =  ghbdelta_spd.values[botm[zxy[:,0],zxy[:,1],zxy[:,2]]<0]
    # join dataframes of 3 ghb boundaries together
    ghb_spd = np.vstack((ghbdn_spd, ghbse_spd.values, ghbnw_spd.values))


    # allocate empty dictionary
    ghb_dict = {}
    ghb_dict[0] = ghb_spd

    # create GHB for flopy
    ghb = flopy.modflow.ModflowGhb(model = m,stress_period_data =  ghb_dict)
    # overwrite the previous ghb file with updated version
    ghb.write_file()

    print('GHB done')


    ###############################################################################
    ## Update LAK Package ##
    if basename(all_model_ws)=='setback_streamflow':
        lak = m.lak
        lak_grid_clip = gpd.read_file(proj_dir+'lak_grid_clip/lak_grid_clip.shp')

        # look at shallower TPROGs data, should be near conductivity of unsat zone
        bdlknc = np.zeros(( lak.bdlknc.shape))

        tprogs_info = [80, -80, 320]
        lak_rows = (lak_grid_clip.row-1).values
        lak_cols = (lak_grid_clip.column-1).values
        lkbd_thick = 4
        top = m.dis.top.array
        bot_str_arr = m.dis.top.array- lkbd_thick
        # get_tprogs_for_elev(K, m_c.dis.top.array, m_c.dis.top.array- np.linspace(1,4,m_c.dis.ncol), rows = sfr_rows, cols = sfr_cols)
        lkbd_tprogs = tc.get_tprogs_for_elev(K, top, bot_str_arr, tprogs_info,rows = lak_rows, cols = lak_cols)
        lk_K = gmean(lkbd_tprogs,axis=0)/100 # divide by 100 to ease convergence, but variability is still there
        #     # set blodgett dam Ksat same as stream Ksat at same location, leakance is K/lakebed thickness
        #     lkbd_thick = sfr.reach_data.strthick[XSg.loc[XSg.Site==16.5].reach]
        bdlknc[:,:,lak_rows,lak_cols] = lk_K/lkbd_thick

        lak.bdlknc = bdlknc

        lak.write_file()

        print('LAK done')


    ###############################################################################
    ## Run the model ##

    # run the modflow model
#     success, buff = m.run_model()